In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("../data/无经纬度的名单.xlsx")
df.head()

,公司1,经度,纬度
0,北京大学,NaN,NaN
1,四川大学,NaN,NaN
2,东南大学,NaN,NaN
3,中南大学,NaN,NaN
4,中山大学,NaN,NaN


In [3]:
import json
from typing import Dict


def save_json(data: Dict, file: str):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def load_json(file) -> Dict:
    with open(file, "r", encoding="utf-8") as f:
        return json.load(f)

baidu_json_data = load_json("../API_results/无经纬度的名单.json")

In [4]:
baidu_json_data["北京市西城区金融大街31号"]

{'status': 0,
 'result': {'location': {'lng': 116.36452971128516, 'lat': 39.91740572650949},
  'precise': 1,
  'confidence': 80,
  'comprehension': 100,
  'level': '门址'},
 'idx': 6,
 'address_input': '北京市西城区金融大街31号'}

In [5]:
def query_data(company_name):
    # 创建一个游标对象
    global baidu_json_data
    res = baidu_json_data.get(company_name, {})
    if not res:
        return pd.NA, pd.NA
    # 获取查询结果
    obj = res["result"]["location"]
    return obj["lng"], obj["lat"]

In [6]:
def add_lat_long(row):
    company_name = row["公司1"]
    long, lat = query_data(company_name)
    return pd.Series({"经度": long, "纬度": lat})

In [7]:
df[["经度", "纬度"]] = df.apply(add_lat_long, axis=1)

In [8]:
df.head()

,公司1,经度,纬度
0,北京大学,116.363768,39.764275
1,四川大学,104.084036,30.630194
2,东南大学,118.782056,32.080824
3,中南大学,112.998124,28.145605
4,中山大学,113.9579,22.804672


In [9]:
df.to_csv("无经纬度的名单_update.csv", index=False)